# Integrate annotated TIL & COLON

## Load adata annotated 

In [ ]:
# Libraries
import anndata as ad
import matplotlib as plt
import numpy as np
import pandas as pd
import sc_atlas_helpers as ah
import scanpy as sc
from matplotlib.pyplot import rc_context
from scipy.stats import median_abs_deviation

In [ ]:
from functools import partial

import altair as alt

In [ ]:
import seaborn as sns

In [ ]:
import decoupler as dc

In [ ]:
resDir = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/"
inputDir = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/"

In [ ]:
adata= sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/adata_merged_til_slec_mpec_annotation_tryneighbors.h5ad")

In [ ]:
path = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/solo"

In [ ]:
adata_solo = sc.read_h5ad(f"{path}/adata_nodoublet.h5ad")

### Functions

In [ ]:
def update_columns(row):
   
    if "ICI" not in row['sample_id']:
        row['origin'] = "colon"
    else:
         row['origin'] = "til"
        
    return row



In [ ]:
def update_columns(row):
   
    if "ICI1"  in row['sample_id']:
        row['batch_id'] = "ICI1"
    elif "ICI2" in row['sample_id']:
         row['batch_id'] = "ICI2"
    elif row['sample_id'] == "10mix1":
        row['batch_id'] = "1"
    elif row['sample_id'] == "10mix2":
         row['batch_id'] = "2"
        
        
    return row



In [ ]:
def update_columns(row):
   
    if row['sample_id'] == "10mix1":
        row['condition'] = "10mix"
    elif row['sample_id'] == "10mix2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix2":
         row['condition'] = "11mix"
    elif row['sample_id'] == "GF1":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI2-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI1-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI1":
         row['condition'] = "GF"
    elif row['sample_id'] == "10mix-ICI1":
         row['condition'] = "10mix"
    elif row['sample_id'] == "10mix-ICI2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix-ICI1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix-ICI2":
         row['condition'] = "11mix"
    
    
        
        
    return row



### Map adata cell type column to adata_solo based on cell barcodes 

In [ ]:
adata_solo.obs["sample_id"]  = adata_solo.obs["sample_id"].str.replace("-","_")

In [ ]:
new_obs_names = adata_solo.obs_names.str.replace(r"-.+", "-1", regex=True)

# Step 2: Concatenate with "_" and adata_solo.obs["sample_id"]
new_obs_names += "_" + adata_solo.obs["sample_id"].astype(str)

# Assign the new values back to adata_solo.obs_names
adata_solo.obs_names = new_obs_names

In [ ]:
cell_type_data = adata.obs["cell_type"]
adata_solo.obs["cell_type"] = adata_solo.obs.index.map(cell_type_data)

In [ ]:
sc.pl.umap(adata_solo, color=["cell_type"], vmax="p99", cmap="Reds")

In [ ]:
# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
sc.pl.umap(adata_solo, color=["cell_type","origin"])

### Split adata

In [ ]:
adata_solo_til = adata_solo[adata_solo.obs["origin"] == "til"]

In [ ]:
adata_solo_colon = adata_solo[adata_solo.obs["origin"] == "colon"]

### Annotate colon

In [ ]:
sc.pl.umap(adata_solo_colon, color=["condition","sample_id"])

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Itga4"], vmax="p99", cmap="Reds")

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Cd69","Il2ra"], vmax="p99", cmap="Reds")

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Sell","Cd44"], vmax="p99", cmap="Reds")

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Klrg1","Il7r","Tbx21"], vmax="p99", cmap="Reds")

In [ ]:
sc.tl.rank_genes_groups(adata_solo_colon, "leiden", method="t-test")

In [ ]:
pd.DataFrame(adata_solo_colon.uns["rank_genes_groups"]["names"]).head(20)

In [ ]:
adata_solo_colon.var["ribo"] = adata_solo_colon.var_names.str.startswith(("Rps", "Rpl"))

In [ ]:
adata_solo_colon = adata_solo_colon[:, adata_solo_colon.var['ribo'] == False].copy()

In [ ]:
adata_solo_colon

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Cd7","Lef1","Ccl5","Il31ra","Ifngr1","Ifng"], vmax="p99", cmap="Reds")

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Ifng","Tox","Itgae","Mki67","Itga4"], vmax="p99", cmap="Reds")

In [ ]:
adata_solo_colon

In [ ]:
sc.pl.umap(adata_solo_colon, color=["leiden"],vmax="p99", cmap="Reds")

In [ ]:
sc.pl.umap(adata_solo_colon, color=["Sell","Cd44"], vmax="p99", cmap="Reds")

In [ ]:
# Adata til2

In [ ]:
sc.pp.neighbors(adata_solo_colon, n_neighbors=10)
sc.tl.leiden(adata_solo_colon, resolution=0.5,  n_iterations=-1)

In [ ]:
sc.pl.umap(adata_solo_colon, color=["leiden"],vmax="p99", cmap="Reds")